**Задание 2: Работа с датасетами (30 баллов)**\
**2.1 Кастомный Dataset класс (15 баллов)**

In [5]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np

class CustomCSVDataset(Dataset):
    def __init__(self, filepath, target_col):
        self.data = pd.read_csv(filepath) #Загружаем CSV-файл

        self.data = self.data.dropna() #удаление строк с пропусками

        self.X = self.data.drop(columns=[target_col]) #Отделяем признаки
        self.y = self.data[target_col] #целевая

        cat_cols = self.X.select_dtypes(include='object').columns #Находим категориальные признаки

        #Кодируем категориальные признаки
        for col in cat_cols:
            self.X[col] = LabelEncoder().fit_transform(self.X[col])

        num_cols = self.X.drop(columns=cat_cols).select_dtypes(include='number').columns #Находим числовые признаки (исключая категориальные)

        #Нормализуем числовые признаки
        scaler = StandardScaler()
        self.X[num_cols] = scaler.fit_transform(self.X[num_cols])

        # Если задача регрессии — нормализуем и y
        if self.y.nunique() > 2:
            self.y_scaler = StandardScaler()
            self.y = self.y_scaler.fit_transform(self.y.values.reshape(-1, 1)).flatten()
            self.is_regression = True
        else:
            self.y = LabelEncoder().fit_transform(self.y)
            self.is_regression = False

        self.X = torch.tensor(self.X.values, dtype=torch.float32)
        self.y = torch.tensor(np.array(self.y), dtype=torch.float32 if self.is_regression else torch.long)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

**2.2 Эксперименты с различными датасетами (15 баллов)**

**Регрессия**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from utils import log_epoch

# Класс линейной регрессии
class LinearRegression(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.linear = nn.Linear(in_features, 1)

    def forward(self, x):
        return self.linear(x).squeeze(1)

# Загружаем датасет
dataset = CustomCSVDataset(filepath="car_data.csv", target_col="price_usd")
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Инициализируем модель
model = LinearRegression(in_features=dataset.X.shape[1])
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Обучение модели
epochs = 100
for epoch in range(1, epochs + 1):
    total_loss = 0
    for batch_X, batch_y in dataloader:
        optimizer.zero_grad()
        y_pred = model(batch_X)
        loss = criterion(y_pred, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(dataloader)
    if epoch % 10 == 0:
        log_epoch(epoch, avg_loss)

Epoch 10: loss=0.2341
Epoch 20: loss=0.2340
Epoch 30: loss=0.2337
Epoch 40: loss=0.2341
Epoch 50: loss=0.2344
Epoch 60: loss=0.2346
Epoch 70: loss=0.2331
Epoch 80: loss=0.2341
Epoch 90: loss=0.2342
Epoch 100: loss=0.2338


**Бинарная классификация**

In [21]:
class LogisticRegression(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.linear = nn.Linear(in_features, 1)

    def forward(self, x):
        return torch.sigmoid(self.linear(x)).squeeze(1)

# Загружаем датасет
dataset = CustomCSVDataset(filepath="water_potability.csv", target_col="Potability")
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Инициализируем модель
model = LogisticRegression(in_features=dataset.X.shape[1])
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.05)

# Обучение модели
epochs = 100
for epoch in range(1, epochs + 1):
    total_loss = 0
    for batch_X, batch_y in dataloader:
        optimizer.zero_grad()
        y_pred = model(batch_X)
        loss = criterion(y_pred, batch_y.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(dataloader)
    if epoch % 10 == 0:
        log_epoch(epoch, avg_loss)


Epoch 10: loss=0.6743
Epoch 20: loss=0.6741
Epoch 30: loss=0.6745
Epoch 40: loss=0.6740
Epoch 50: loss=0.6740
Epoch 60: loss=0.6741
Epoch 70: loss=0.6741
Epoch 80: loss=0.6743
Epoch 90: loss=0.6742
Epoch 100: loss=0.6740
